In [1]:
import os, json, io
import numpy as np
import pandas as pd
# from models.load import TrainedModels

from utils.eval import save_iou_results
from utils.engine import get_iou_types, evaluate
from models.load import get_trained_model
from utils.eval import get_ap_ar, get_num_fps, get_num_fns, get_num_tps
from utils.print import print_title
from utils.init import reproducibility, clean_memory_get_device
from data.load import get_datasets, get_dataloaders
from data.paths import MIMIC_EYE_PATH
from tqdm import tqdm
from utils.train import  get_coco_eval_params
from utils.coco_eval import get_eval_params_dict
from data.strs import SourceStrs, TaskStrs

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
import itertools
from scipy import interpolate

def get_froc_curve(dataset, dts, eval_gts):
    anns = []
    gts = [
        g["lesion-detection"]
        for g in list(itertools.chain.from_iterable(list(eval_gts)))
    ]
    for g in gts:
        for i, bb in enumerate(g["unsized_boxes"]):
            anns.append(
                {
                    "image_id": g["image_id"].item(),
                    "category_id": g["labels"][i].item(),
                    "iscrowd": g["iscrowd"][i].item(),
                    "bbox": bb.tolist(),
                    "ignore": 0,
                }
            )

    pr = []
    for dt in dts:
        for image_id, d in dt.items():
            for i in range(len(d["boxes"])):
                pr.append(
                    {
                        "image_id": image_id,
                        "category_id": d["labels"][i].item(),
                        "score": d["scores"][i].item(),
                        "bbox": d["boxes"][i].numpy().tolist(),
                    }
                )

    categories = [
        {
            "id": dataset.disease_to_idx(i),
            "name": i,
            "color": "blue",
        }
        for i in dataset.labels_cols
    ]

    gt = {
        "categories": categories,
        "annotations": anns,
        "images": [{"id": id} for id in list(set([g["image_id"].item() for g in gts]))],
    }

    from coco_froc_analysis.froc.froc_curve import generate_froc_curve

    lls_accuracy, nlls_per_image = generate_froc_curve(
        gt=gt,
        pr=pr,
        use_iou=True,
        iou_thres=0.5,
        n_sample_points=100,
        plot_title=None,
        plot_output_path="froc.png",
        bounds=None,
    )

    return lls_accuracy, nlls_per_image


In [3]:
def get_interpolate_froc(
    lls_accuracy,nlls_per_image, cat_id=-1, fps_per_img=[0.5, 1, 2, 4]
):
    f = interpolate.interp1d(
        nlls_per_image[cat_id], lls_accuracy[cat_id], fill_value="extrapolate"
    )
    return f(fps_per_img)


In [4]:
from enum import Enum

class TrainedModels(Enum):
    mobilenet_baseline = "val_lesion-detection_ap_0_1655_test_lesion-detection_ap_0_1648_epoch50_03-15-2023 16-43-54_lesion_dsetection_baseline_mobilenet"  # mobilenet baseline
    mobilenet_with_fix = "val_lesion-detection_ap_0_1918_test_lesion-detection_ap_0_1903_epoch16_03-16-2023 11-34-10_lesion_dsetection_with_fixation_mobilenet"
    resnet18_baseline = "val_lesion-detection_ap_0_1973_test_lesion-detection_ap_0_2010_epoch22_03-16-2023 19-44-55_lesion_dsetection_baseline_resnet"
    resnet18_with_fix = "val_lesion-detection_ap_0_1951_test_lesion-detection_ap_0_2195_epoch12_03-17-2023 00-31-54_lesion_dsetection_with_fixation_resnet"
    densenet161_baseline = "val_lesion-detection_ap_0_1990_test_lesion-detection_ap_0_2085_epoch5_03-17-2023 08-53-33_lesion_dsetection_baseline_densenet161"
    densenet161_with_fix = "val_lesion-detection_ap_0_2120_test_lesion-detection_ap_0_2104_epoch12_03-17-2023 18-36-01_lesion_dsetection_with_fixation_densenet161"
    efficientnet_b5_baseline = "val_lesion-detection_ap_0_1898_test_lesion-detection_ap_0_2055_epoch5_03-17-2023 23-30-57_lesion_dsetection_baseline_efficientnet_b5"
    efficientnet_b5_with_fix = "val_lesion-detection_ap_0_2117_test_lesion-detection_ap_0_2190_epoch8_03-18-2023 12-29-20_lesion_dsetection_with_fixation_efficientnet_b5"
    efficientnet_b0_baseline = "val_lesion-detection_ap_0_1934_test_lesion-detection_ap_0_1858_epoch10_03-18-2023 23-50-47_lesion_dsetection_baseline_efficientnet_b0"
    efficientnet_b0_with_fix = "val_lesion-detection_ap_0_2191_test_lesion-detection_ap_0_2162_epoch10_03-18-2023 19-38-11_lesion_dsetection_with_fixation_efficientnet_b0"
    convnext_base_with_fix = "val_lesion-detection_ap_0_2610_test_lesion-detection_ap_0_2548_epoch22_03-22-2023 02-55-37_lesion_dsetection_with_fixation_convnext_base"
    convnext_base_baseline = "val_lesion-detection_ap_0_2426_test_lesion-detection_ap_0_2325_epoch20_03-22-2023 11-53-53_lesion_dsetection_baseline_convnext_base"
    convnext_base_with_fix_silent = "val_lesion-detection_ap_0_2405_test_lesion-detection_ap_0_2543_epoch19_03-24-2023 13-57-29_lesion_dsetection_with_fixation_convnext_base_silent_report"
    convnext_base_with_fix_full = "val_lesion-detection_ap_0_2602_test_lesion-detection_ap_0_2499_epoch22_03-24-2023 04-42-21_lesion_dsetection_with_fixation_convnext_base_full_report"

    vgg16_with_fix = "val_lesion-detection_ap_0_2301_test_lesion-detection_ap_0_2186_epoch22_03-20-2023 19-26-02_lesion_dsetection_with_fixation_vgg16"
    vgg16_baseline = "val_lesion-detection_ap_0_2113_test_lesion-detection_ap_0_2068_epoch12_03-21-2023 00-45-24_lesion_dsetection_baseline_vgg16"
    regnet_y_8gf_with_fix = "val_lesion-detection_ap_0_2267_test_lesion-detection_ap_0_2029_epoch12_03-21-2023 11-28-48_lesion_dsetection_with_fixation_regnet_y_8gf"
    regnet_y_8gf_baseline = "val_lesion-detection_ap_0_1883_test_lesion-detection_ap_0_1658_epoch13_03-21-2023 15-22-32_lesion_dsetection_baseline_regnet_y_8gf"

In [5]:
normal_iou_thrs = np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])
score_thresholds = [0.05]

In [6]:
# just generate for the test set.
for select_model in tqdm(list(TrainedModels)):

    for score_thrs in score_thresholds:

        device = clean_memory_get_device()
        reproducibility()

        model, train_info, _, _ = get_trained_model(
            select_model,
            device,
        )
        model = model.to(device)
        model.eval()

        setup = train_info.model_setup
        iou_types = get_iou_types(model, setup)

        dataset_params_dict = {
            "MIMIC_EYE_PATH": MIMIC_EYE_PATH,
            "labels_cols": setup.lesion_label_cols,
            "with_xrays_input": SourceStrs.XRAYS in setup.sources,
            "with_clincal_input": SourceStrs.CLINICAL in setup.sources,
            "with_fixations_input": SourceStrs.FIXATIONS in setup.sources,
            "fixations_mode_input": setup.fiaxtions_mode_input,
            "with_bboxes_label": TaskStrs.LESION_DETECTION in setup.tasks,
            "with_fixations_label": TaskStrs.FIXATION_GENERATION in setup.tasks,
            "fixations_mode_label": setup.fiaxtions_mode_label,
            "with_chexpert_label": TaskStrs.CHEXPERT_CLASSIFICATION in setup.tasks,
            "with_negbio_label": TaskStrs.NEGBIO_CLASSIFICATION in setup.tasks,
            "clinical_numerical_cols": setup.clinical_num,
            "clinical_categorical_cols": setup.clinical_cat,
            "image_size": setup.image_size,
            "image_mean": setup.image_mean,
            "image_std": setup.image_std,
        }

        detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
            dataset_params_dict=dataset_params_dict,
        )

        train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
            train_dataset,
            val_dataset,
            test_dataset,
            batch_size=setup.batch_size,
        )

        train_coco = None
        train_coco, val_coco, test_coco, _ = get_coco_eval_params(
            source_name=SourceStrs.XRAYS,
            task_name=TaskStrs.LESION_DETECTION,
            train_dataloader=train_dataloader,
            val_dataloader=val_dataloader,
            test_dataloader=test_dataloader,
            detect_eval_dataset=detect_eval_dataset,
            iou_thrs=np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]),
            use_iobb=setup.use_iobb,
            maxDets=[1, 5, 10, 30],
            # train_dataloader,
            # val_dataloader,
            # test_dataloader,
            # detect_eval_dataset,
            # setup.iou_thrs,
            # setup.use_iobb,
        )

        normal_range_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset,
            iou_thrs=normal_iou_thrs,
        )

        model.task_performers["lesion-detection"].roi_heads.score_thresh = 0
        test_evaluator, _, (dts, eval_gts) = evaluate(
            setup=setup,
            model=model,
            data_loader=test_dataloader,
            device=device,
            params_dict=normal_range_eval_params_dict,
            coco=test_coco,
            iou_types=iou_types,
            return_dt_gt=True,
            # score_thres=score_thres,
        )
        model.task_performers["lesion-detection"].roi_heads.score_thresh = 0.05

        lls_accuracy, nlls_per_image = get_froc_curve(
            dataset=test_dataset,
            dts=dts,
            eval_gts=eval_gts,
        )

        all_cat_ids = [None] + [
            detect_eval_dataset.disease_to_idx(d)
            for d in detect_eval_dataset.labels_cols
        ]

        for cat_id in all_cat_ids:
            cat_ids = (
                [
                    detect_eval_dataset.disease_to_idx(d)
                    for d in detect_eval_dataset.labels_cols
                ]
                if cat_id is None
                else [cat_id]
            )

            if not (cat_ids is None):
                normal_range_eval_params_dict["bbox"].catIds = cat_ids

            test_evaluator, _ = evaluate(
                setup=setup,
                model=model,
                data_loader=test_dataloader,
                device=device,
                params_dict=normal_range_eval_params_dict,
                coco=test_coco,
                iou_types=iou_types,
            )

            if cat_id is None:
                disease_str = "all"
            else:
                disease_str = detect_eval_dataset.label_idx_to_disease(cat_id)

            # disease_str = detect_eval_dataset.label_idx_to_disease(cat_id)

            # save_iou_results(
            #     test_evaluator['lesion-detection'],
            #     f"test_{disease_str}_score_thrs{score_thrs}",
            #     select_model.value,
            # )

            # test_ap_ar = get_ap_ar(
            #     test_evaluator['lesion-detection'],
            #     areaRng="all",
            #     maxDets=test_evaluator['lesion-detection'].coco_eval['bbox'].params.maxDets[-1],
            #     iouThr=None,
            # )

            # test_ap_ar = get_ap_ar(
            #     test_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            # )
            # val_ap_ar = get_ap_ar(val_evaluator, areaRng="all", maxDets=10, iouThr=0.5,)

            # df = pd.DataFrame(
            #     [
            #         {
            #             f"AP@[.50:.05:.95]": test_ap_ar["ap"],
            #             f"AR@[.50:.05:.95]": test_ap_ar["ar"],
            #         },
            #     ]
            # )

            # df.to_csv(
            #     os.path.join(
            #         "eval_results",
            #         f"{select_model.value}_{disease_str}_score_thrs{score_thrs}.csv",
            #     )
            # )

            # num_fps = get_num_fps(
            #     test_evaluator["lesion-detection"],
            #     areaRng="all",
            #     maxDets=10,
            #     iouThr=0.5,
            # )

            # num_fns = get_num_fns(
            #     test_evaluator["lesion-detection"],
            #     areaRng="all",
            #     maxDets=10,
            #     iouThr=0.5,
            # )

            # num_tps = get_num_tps(
            #     test_evaluator["lesion-detection"],
            #     areaRng="all",
            #     maxDets=10,
            #     iouThr=0.5,
            # )

            # coco_eval_str = ""
            # for state in test_evaluator['lesion-detection'].coco_eval['bbox'].stats:
            #     coco_eval_str += f"{get_ap_ar_str(state,test_evaluator['lesion-detection'].coco_eval['bbox'].params)}\n"

            froc_v = get_interpolate_froc(
                lls_accuracy=lls_accuracy,
                nlls_per_image=nlls_per_image,
                cat_id=-1 if cat_id is None else cat_id,
                fps_per_img=[0.5, 1, 2, 4],
            )

            df = pd.DataFrame(
                [
                    {
                        "num_fps": test_evaluator["lesion-detection"]
                        .coco_eval["bbox"]
                        .eval["num_fps"],  # @all_range, maxDet= p.maxDets[-1]
                        "num_fns": test_evaluator["lesion-detection"]
                        .coco_eval["bbox"]
                        .eval["num_fns"],
                        "num_tps": test_evaluator["lesion-detection"]
                        .coco_eval["bbox"]
                        .eval["num_tps"],
                        "coco_states": json.dumps(
                            test_evaluator["lesion-detection"].coco_eval["bbox"].stats
                        ),
                        "Sensitivity@ [avgFP=0.5]": froc_v[0],
                        "Sensitivity@ [avgFP=1]": froc_v[1],
                        "Sensitivity@ [avgFP=2]": froc_v[2],
                        "Sensitivity@ [avgFP=4]": froc_v[3],
                        "mFROC@[0.5,1,2,4]": froc_v.mean(),
                    },
                ]
            )

            # df = pd.DataFrame(
            #     [
            #         {
            #             "num_fps": test_evaluator['lesion-detection'].coco_eval['bbox'].eval['num_fps'], # @all_range, maxDet= p.maxDets[-1]
            #             "num_fns": test_evaluator['lesion-detection'].coco_eval['bbox'].eval['num_fns'],
            #             "num_tps": test_evaluator['lesion-detection'].coco_eval['bbox'].eval['num_tps'],
            #             # f"AP@[.50:.05:.95]": test_ap_ar["ap"],
            #             # f"AR@[.50:.05:.95]": test_ap_ar["ar"],
            #             "coco_states": json.dumps(test_evaluator['lesion-detection'].coco_eval['bbox'].stats)
            #         },
            #     ]
            # )

            df.to_csv(
                os.path.join(
                    "./eval_results",
                    f"{select_model.value}_{disease_str}_{score_thrs}.csv",
                )
            )

            print_title(f"{select_model.value}-{disease_str}")
            print(df)


  0%|          | 0/18 [00:00<?, ?it/s]

This notebook will running on device: [CUDA]
Using pretrained backbone. mobilenet_v3
Using SGD as optimizer with lr=0.01
